# Static Load Calculations for Snow Slabs

This notebook demonstrates methods for calculating the static load (weight per unit area) of layered snow slabs and their force components on slopes using the `snowpyt_mechparams` package.

## Overview

The static load calculations include:
- **Gravitational load**: Total weight per unit area (vertical force)
- **Shear load**: Force component parallel to the slope surface
- **Normal load**: Force component perpendicular to the slope surface

These calculations are essential for avalanche hazard assessment and snow stability analysis.


In [3]:
# Import Libraries
import os
import sys

import pandas as pd

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpyt_mechparams import density
from snowpyt_mechparams.data_structures import Layer, Slab
from snowpyt_mechparams.static_load import calculate_static_load


## Parse Snowpit Files

Parse CAAML snowpit files to extract real layer data:


In [4]:
all_pits = []
failed_files = []

folder_path = 'data'

xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

for file in xml_files:
    try:
        file_path = os.path.join(folder_path, file)
        pit = caaml_parser(file_path)
        all_pits.append(pit)
    except Exception as e:
        failed_files.append((file, str(e)))
        print(f"Warning: Failed to parse {file}: {e}")

print(f"Successfully parsed {len(all_pits)} files")
print(f"Failed to parse {len(failed_files)} files")


Successfully parsed 50278 files
Failed to parse 0 files


## Construct Slab

In [8]:
# For pits with indicated layer of concern

slabs = []

for pit in all_pits:
    layers = pit.snow_profile.layers
    if pit.core_info.location.slope_angle is not None and any(getattr(layer, 'layer_of_concern', False) for layer in layers): # if slope angle exists and any layer of concern is indicated
        slab_layers = []
        for layer in layers:
            if layer.layer_of_concern:
                break
            hardness = layer.hardness
            primary_grain_form = layer.grain_form_primary
            density = density.get_density(hardness, primary_grain_form)
            layer_obj = Layer(depth_from_surface=layer.depth_from_surface, density=density)
            slab_layers.append(layer_obj)
        slab = Slab(layers=slab_layers, angle=pit.core_info.location.slope_angle)


print(f"Found {len(slabs)} pits with slope angle and layer of concern")



AttributeError: module 'snowpyt_mechparams.density' has no attribute 'get_density'

## Static Load Calculations

